<a href="https://colab.research.google.com/github/zloelias/final-project/blob/master/character_level_lenta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#! pip install tensorflow_text
#! pip install textattack['tensorflow']
#! pip install datasets --upgrade
#! pip install captum

In [ ]:
# При установке из репозитория при работе не подцепляются другие модели, работает только дефолтный bert-base-uncased

! pip install tensorflow_text
! git clone https://github.com/zloelias/TextAttack.git
! pip install /content/TextAttack[tensorflow] --use-feature=in-tree-build
! pip install datasets --upgrade
! pip install captum

fatal: destination path 'TextAttack' already exists and is not an empty directory.
Processing ./TextAttack
  Created wheel for textattack: filename=textattack-0.3.4-py3-none-any.whl size=377434 sha256=cec3290e17a0e683caef42d35287b4e3464f06a6f2bf96f12dc519f2b60796f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-mwohwodq/wheels/00/07/19/705a89c140a25ea4a53610153f7b1440b6c62115ea7904bdc6
Successfully built textattack
  Attempting uninstall: textattack
    Found existing installation: textattack 0.3.4
    Uninstalling textattack-0.3.4:
      Successfully uninstalled textattack-0.3.4


In [ ]:
import torch
from copy import deepcopy

from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper
from transformers import AutoModelForSequenceClassification, AutoTokenizer

#from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients, LayerDeepLiftShap, InternalInfluence, LayerGradientXActivation
#from captum.attr import visualization as viz

import nltk
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else: 
    device = torch.device("cpu")
    
print(device)

cuda:0


In [ ]:
from textattack import Attack
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder, MultilingualUniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR, GreedySearch
from textattack.transformations import (
    CompositeTransformation,
    #WordSwapEmbedding,
    WordSwapHomoglyphSwap,
    WordSwapQWERTY,
    WordSwapNeighboringCharacterSwap,
    WordSwapRandomCharacterDeletion,
    WordSwapRandomCharacterInsertion,
)

from textattack.constraints.pre_transformation.max_word_index_modification import MaxWordIndexModification
from textattack.constraints.pre_transformation.max_modification_rate import MaxModificationRate

from textattack.attack_recipes.attack_recipe import AttackRecipe


In [ ]:
class TextBuggerLi2018RU(AttackRecipe):
    """Li, J., Ji, S., Du, T., Li, B., and Wang, T. (2018).
    TextBugger: Generating Adversarial Text Against Real-world Applications.
    https://arxiv.org/abs/1812.05271
    """

    @staticmethod
    def build(model_wrapper, max_rate, max_seq_len, language):
        #
        #  we propose five bug generation methods for TEXTBUGGER:
        #
        transformation = CompositeTransformation(
            [
                # (1) Insert: Insert a space into the word.
                # Generally, words are segmented by spaces in English. Therefore,
                # we can deceive classifiers by inserting spaces into words.
                WordSwapRandomCharacterInsertion(
                    random_one=True,
                    letters_to_insert=" ",
                    skip_first_char=True,
                    skip_last_char=True,
                ),
                # (2) Delete: Delete a random character of the word except for the first
                # and the last character.
                WordSwapRandomCharacterDeletion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                # (3) Swap: Swap random two adjacent letters in the word but do not
                # alter the first or last letter. This is a common occurrence when
                # typing quickly and is easy to implement.
                WordSwapNeighboringCharacterSwap(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                # (4) Substitute-C (Sub-C): Replace characters with visually similar
                # characters (e.g., replacing “o” with “0”, “l” with “1”, “a” with “@”)
                # or adjacent characters in the keyboard (e.g., replacing “m” with “n”).
                WordSwapHomoglyphSwap(),
                # (5) Substitute-W
                # (Sub-W): Replace a word with its topk nearest neighbors in a
                # context-aware word vector space. Specifically, we use the pre-trained
                # GloVe model [30] provided by Stanford for word embedding and set
                # topk = 5 in the experiment.
                #WordSwapEmbedding(max_candidates=5),
                WordSwapQWERTY(random_one=True, skip_first_char=True, skip_last_char=True),
            ]
        )

        constraints = [RepeatModification(), StopwordModification(language=language)]
        constraints.append(MaxWordIndexModification(max_seq_len))
        constraints.append(MaxModificationRate(max_rate, min_threshold=1))

        # In our experiment, we first use the Universal Sentence
        # Encoder [7], a model trained on a number of natural language
        # prediction tasks that require modeling the meaning of word
        # sequences, to encode sentences into high dimensional vectors.
        # Then, we use the cosine similarity to measure the semantic
        # similarity between original texts and adversarial texts.
        # ... "Furthermore, the semantic similarity threshold \eps is set
        # as 0.8 to guarantee a good trade-off between quality and
        # strength of the generated adversarial text."
        constraints.append(MultilingualUniversalSentenceEncoder(
            threshold=0.936338023,
            metric="cosine",
            compare_against_original=True,
            window_size=15,
            skip_text_shorter_than_window=True,
        ))
        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return Attack(goal_function, constraints, transformation, search_method)




In [ ]:
import textattack
import random
import numpy as np

experiments = {
    'simple_char_level_attack': [
#        {
#            'dataset': {'name': 'kinopoisk', 'url': 'zloelias/kinopoisk-reviews'},
#            'model': ('cointegrated/rubert-tiny2', 'zloelias/rubert-tiny2-kinopoisk-reviews-finetuned-clf'),
#            'language': ('ru', 'rus', 'russian'),
#            'max_rate': np.linspace(0.1, 0.9, 9),
#            'max_seq_len': 312,
#            'columns': (['text'], 'labels'),
#        },
        {
            'dataset': {'name': 'lenta', 'url': 'zloelias/lenta-ru'},
            'model': ('cointegrated/rubert-tiny2', 'zloelias/rubert-tiny2-lenta-ru-finetuned-clf'),
            'language': ('ru', 'rus', 'russian'),
            'max_rate': np.linspace(0.1, 0.9, 9),
            'max_seq_len': 312,
            'columns': (['text'], 'labels'),
        },
#        {
#            'dataset': {'name': 'ag_news', 'url': 'ag_news'},
#            'model': ('bert-base-uncased', 'textattack/bert-base-uncased-ag-news'),
#            'language': ('en', 'eng', 'english'),
#            'max_rate': np.linspace(0.1, 0.9, 9),
#            'max_seq_len': 512,
#            'columns': (['text'], 'label'),
#        },
#        {
#            'dataset': {'name': 'rotten_tomatoes', 'url': 'rotten_tomatoes'},
#            'model': ('bert-base-uncased', 'textattack/bert-base-uncased-rotten-tomatoes'),
#            'language': ('en', 'eng', 'english'),
#            'max_rate': np.linspace(0.1, 0.9, 9),
#            'max_seq_len': 512,
#            'columns': (['text'], 'label'),
#        },
    ]
}

for attack_name in experiments.keys():
    for attack_data in experiments[attack_name]:
        dataset_name, dataset_url = [attack_data['dataset'][k] for k in attack_data['dataset'].keys()]
        dataset_columns = attack_data['columns']
        dataset = HuggingFaceDataset(dataset_url, None, 'test', dataset_columns=dataset_columns)

        base_model_name, model_name = attack_data['model']
        original_model = AutoModelForSequenceClassification.from_pretrained(model_name)
        original_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
        model = HuggingFaceModelWrapper(original_model,original_tokenizer)

        la, lan, language = attack_data['language']
        for max_rate in attack_data['max_rate']:
            experiment_result_dir = f'/content/drive/MyDrive/Colab Notebooks/final-project/experiments/{attack_name}/{dataset_name}/{max_rate}/'
            print(experiment_result_dir)
            attack = TextBuggerLi2018RU.build(model, max_rate, attack_data['max_seq_len'], language)

            attack_args = textattack.AttackArgs(
                num_examples=300, 
                log_to_csv=experiment_result_dir + "log.csv", 
                checkpoint_interval=100, 
                checkpoint_dir=experiment_result_dir + "checkpoints", 
                disable_stdout=True
            )
            attacker = textattack.Attacker(attack, dataset, attack_args)
            attacker.attack_dataset()
        #print(attack_data)
        #print(model_name, model_url)

Using custom data configuration zloelias___lenta-ru-dc5084b443d90e4e
Reusing dataset parquet (/root/.cache/huggingface/datasets/parquet/zloelias___lenta-ru-dc5084b443d90e4e/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121)


  0%|          | 0/2 [00:00<?, ?it/s]

textattack: Loading datasets dataset zloelias/lenta-ru, split test.
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


/content/drive/MyDrive/Colab Notebooks/final-project/experiments/simple_char_level_attack/lenta/0.1/


textattack: Logging to CSV at path /content/drive/MyDrive/Colab Notebooks/final-project/experiments/simple_char_level_attack/lenta/0.1/log.csv


...





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 49     |
| Number of failed attacks:     | 245    |
| Number of skipped attacks:    | 6      |
| Original accuracy:            | 98.0%  |
| Accuracy under attack:        | 81.67% |
| Attack success rate:          | 16.67% |
| Average perturbed word %:     | 21.03% |
| Average num. words per input: | 181.9  |
| Avg num queries:              | 712.81 |
+-------------------------------+--------+


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path /content/drive/MyDrive/Colab Notebooks/final-project/experiments/simple_char_level_attack/lenta/0.7000000000000001/log.csv



/content/drive/MyDrive/Colab Notebooks/final-project/experiments/simple_char_level_attack/lenta/0.7000000000000001/
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapHomoglyphSwap
    (4): WordSwapQWERTY
    )
  (constraints): 
    (0): MultilingualUniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
    (3): MaxWordIndexModification(
        (max_length):  312
      )
    (4): MaxModificationRate(
        (max

[Succeeded / Failed / Skipped / Total] 12 / 87 / 1 / 100:  33%|███▎      | 100/300 [19:19<38:39, 11.60s/it]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/simple_char_level_attack/lenta/0.7000000000000001/checkpoints/1639450518003.ta.chkpt" at 2021-12-14 02:55:18 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 94 / 1 / 109:  37%|███▋      | 110/300 [21:23<36:57, 11.67s/it]